In [ ]:
import os

from dotenv import load_dotenv
from typing import List, Dict

from pydantic import BaseModel, Field

import google.generativeai as genai

from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool

# import agentops

# from tavily import TavilyClient

ImportError: cannot import name 'ChatGoogle' from 'crewai.llms' (/home/israa/Desktop/Smart_Recruiter_RAG/.venv/lib/python3.10/site-packages/crewai/llms/__init__.py)

In [2]:
load_dotenv()  # Load from .env

False

In [3]:
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

In [ ]:
basic_llm = ChatGoogle(
    model="gemini/gemini-1.5-flash",
    temperature=0.2,
    api_key=GEMINI_API_KEY
)


AttributeError: type object 'LLM' has no attribute 'ChatGoogle'

In [8]:
response = basic_llm.invoke("List top 3 programming languages for data science.")
print(response)

AttributeError: 'LLM' object has no attribute 'invoke'

In [16]:
import os
import fitz  # PyMuPDF
import docx

def extract_text_from_pdf(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def extract_text_from_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

def extract_text_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

def parse_cv(file_path):
    if file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".txt"):
        return extract_text_from_txt(file_path)
    else:
        return None

# Example: Parse all CVs in a folder
folder_path = "../data/"
parsed_cvs = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    text = parse_cv(file_path)
    if text:
        parsed_cvs.append({"filename": file_name, "text": text})


# for cv in parsed_cvs:
#     print(f"\n--- {cv['filename']} ---")
#     print(cv['text'])  # print only first 1000 characters to keep it short


def chunk_text(text, max_words=200):
    words = text.split()
    return [' '.join(words[i:i+max_words]) for i in range(0, len(words), max_words)]





In [17]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer

client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="./chroma_db"))
collection = client.get_or_create_collection(name="cv_chunks")

model = SentenceTransformer('all-MiniLM-L6-v2')

for file in os.listdir("../data/"):
    path = os.path.join("../data/", file)
    text = parse_cv(path)
    if text:
        chunks = chunk_text(text)
        embeddings = model.encode(chunks).tolist()
        for i, chunk in enumerate(chunks):
            collection.add(
                documents=[chunk],
                embeddings=[embeddings[i]],
                metadatas=[{"filename": file, "chunk": i}],
                ids=[f"{file}_{i}"]
            )


ModuleNotFoundError: No module named 'sentence_transformers'